In [1]:
import os
import gc
import pickle
from datetime import datetime, date, timedelta

import numpy as np
import pandas as pd
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.metrics import mean_absolute_error
from sklearn.compose import TransformedTargetRegressor
from sklearn.ensemble import VotingRegressor
from xgboost import XGBRegressor
import lightgbm as lgb

import optuna

In [2]:
def feature_eng(df_data, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target):
    df_data = (
        df_data
        .with_columns(
            pl.col("datetime").cast(pl.Date).alias("date"),
        )
    )
    
    df_client = (
        df_client
        .with_columns(
            (pl.col("date") + pl.duration(days=2)).cast(pl.Date)
        )
    )
    
    df_gas = (
        df_gas
        .rename({"forecast_date": "date"})
        .with_columns(
            (pl.col("date") + pl.duration(days=1)).cast(pl.Date)
        )
    )
    
    df_electricity = (
        df_electricity
        .rename({"forecast_date": "datetime"})
        .with_columns(
            pl.col("datetime") + pl.duration(days=1)
        )
    )
    
    df_location = (
        df_location
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32)
        )
    )
    
    df_forecast = (
        df_forecast
        .rename({"forecast_datetime": "datetime"})
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_historical = (
        df_historical
        .with_columns(
            pl.col("latitude").cast(pl.datatypes.Float32),
            pl.col("longitude").cast(pl.datatypes.Float32),
        )
        .join(df_location, how="left", on=["longitude", "latitude"])
        .drop("longitude", "latitude")
    )
    
    df_forecast_date = (
        df_forecast
        .group_by("datetime").mean()
        .drop("county")
    )
    
    df_forecast_local = (
        df_forecast
        .filter(pl.col("county").is_not_null())
        .group_by("county", "datetime").mean()
    )
    
    df_historical_date = (
        df_historical
        .filter(pl.col('datetime').dt.hour() <= 10).with_columns(pl.col("datetime") + pl.duration(days=1))
        .with_columns(pl.col("datetime").cast(pl.Date).alias("date"))
        .group_by('date').mean() # .sort('datetime')
        .drop("county","datetime")
    )
    
    df_historical_local = (
        df_historical
        .filter(pl.col('datetime').dt.hour() <= 10).with_columns(pl.col("datetime") + pl.duration(days=1))
        .with_columns(pl.col("datetime").cast(pl.Date).alias("date"))
        .group_by('date','county').mean() # .sort('date')
        .drop("datetime")
    )

    
    df_data = (
        df_data
        .join(df_gas, on="date", how="left")
        .join(df_client, on=["county", "is_business", "product_type", "date"], how="left")
        .join(df_electricity, on="datetime", how="left")
        
        .join(df_forecast_date, on="datetime", how="left", suffix="_fd")
        .join(df_forecast_local, on=["county", "datetime"], how="left", suffix="_fl")
        # .join(df_historical_date, on="date", how="left", suffix="_hd")
        # .join(df_historical_local, on=["county", "date"], how="left", suffix="_hl")
        
        # .join(df_forecast_date.with_columns(pl.col("datetime") + pl.duration(days=7)), on="datetime", how="left", suffix="_fdw")
        # .join(df_forecast_local.with_columns(pl.col("datetime") + pl.duration(days=7)), on=["county", "datetime"], how="left", suffix="_flw")
        # .join(df_historical_date.with_columns(pl.col("date") + pl.duration(days=7)), on="date", how="left", suffix="_hdw")
        # .join(df_historical_local.with_columns(pl.col("date") + pl.duration(days=7)), on=["county", "date"], how="left", suffix="_hlw")
        
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=2)).rename({"target": "target_1"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=3)).rename({"target": "target_2"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=4)).rename({"target": "target_3"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=5)).rename({"target": "target_4"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=6)).rename({"target": "target_5"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=7)).rename({"target": "target_6"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        .join(df_target.with_columns(pl.col("datetime") + pl.duration(days=14)).rename({"target": "target_7"}), on=["county", "is_business", "product_type", "is_consumption", "datetime"], how="left")
        
        .with_columns(
            pl.col("datetime").dt.ordinal_day().alias("dayofyear"),
            pl.col("datetime").dt.hour().alias("hour"),
            pl.col("datetime").dt.day().alias("day"),
            pl.col("datetime").dt.weekday().alias("weekday"),
            pl.col("datetime").dt.month().alias("month"),
            pl.col("datetime").dt.year().alias("year"),
        )
        
        # .with_columns(
            # pl.concat_str("county", "is_business", "product_type", "is_consumption", separator="_").alias("category_1"),
        # )
        
        .with_columns(
            (np.pi * pl.col("dayofyear") / 183).sin().alias("sin(dayofyear)"),
            (np.pi * pl.col("dayofyear") / 183).cos().alias("cos(dayofyear)"),
            (np.pi * pl.col("hour") / 12).sin().alias("sin(hour)"),
            (np.pi * pl.col("hour") / 12).cos().alias("cos(hour)"),
        )
        
        .with_columns(
            pl.col(pl.Float64).cast(pl.Float32),
        )
        
        .drop("hour", "dayofyear", "datetime")
        
        .with_columns(
            (pl.col("total_precipitation") > 0).cast(pl.datatypes.Int8).alias("is_raining"),
            (pl.col("snowfall") > 0).cast(pl.datatypes.Int8).alias("is_snowing")
        )
    )
    
    return df_data

In [3]:
def to_pandas(X, y=None):
    cat_cols = ["county", "product_type", "is_consumption"] # "category_1", "is_business"
    
    if y is not None:
        df = pd.concat([X.to_pandas(), y.to_pandas()], axis=1)
    else:
        df = X.to_pandas()    
    
    df = df.set_index("row_id")
    df[cat_cols] = df[cat_cols].astype("category")
    
    df["target_mean"] = df[[f"target_{i}" for i in range(1, 7)]].mean(1)
    df["target_std"] = df[[f"target_{i}" for i in range(1, 7)]].std(1)
    df["target_ratio"] = df["target_6"] / (df["target_7"] + 1e-3)
    
    return df

In [4]:
root = "/kaggle/input/predict-energy-behavior-of-prosumers"

data_cols        = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime', 'row_id']
client_cols      = ['product_type', 'county', 'eic_count', 'installed_capacity', 'is_business', 'date']
gas_cols         = ['forecast_date', 'lowest_price_per_mwh', 'highest_price_per_mwh']
electricity_cols = ['forecast_date', 'euros_per_mwh']
forecast_cols    = ['latitude', 'longitude', 'hours_ahead', 'temperature', 'dewpoint', 'cloudcover_high', 'cloudcover_low', 'cloudcover_mid', 'cloudcover_total', '10_metre_u_wind_component', '10_metre_v_wind_component', 'forecast_datetime', 'direct_solar_radiation', 'surface_solar_radiation_downwards', 'snowfall', 'total_precipitation']
historical_cols  = ['datetime', 'temperature', 'dewpoint', 'rain', 'snowfall', 'surface_pressure','cloudcover_total','cloudcover_low','cloudcover_mid','cloudcover_high','windspeed_10m','winddirection_10m','shortwave_radiation','direct_solar_radiation','diffuse_radiation','latitude','longitude']
location_cols    = ['longitude', 'latitude', 'county']
target_cols      = ['target', 'county', 'is_business', 'product_type', 'is_consumption', 'datetime']

save_path = None
load_path = None

In [5]:
df_data        = pl.read_csv(os.path.join(root, "train.csv"), columns=data_cols, try_parse_dates=True)
df_client      = pl.read_csv(os.path.join(root, "client.csv"), columns=client_cols, try_parse_dates=True)
df_gas         = pl.read_csv(os.path.join(root, "gas_prices.csv"), columns=gas_cols, try_parse_dates=True)
df_electricity = pl.read_csv(os.path.join(root, "electricity_prices.csv"), columns=electricity_cols, try_parse_dates=True)
df_forecast    = pl.read_csv(os.path.join(root, "forecast_weather.csv"), columns=forecast_cols, try_parse_dates=True)
df_historical  = pl.read_csv(os.path.join(root, "historical_weather.csv"), columns=historical_cols, try_parse_dates=True)
df_location    = pl.read_csv(os.path.join(root, "weather_station_to_county_mapping.csv"), columns=location_cols, try_parse_dates=True)
df_target      = df_data.select(target_cols)

schema_data        = df_data.schema
schema_client      = df_client.schema
schema_gas         = df_gas.schema
schema_electricity = df_electricity.schema
schema_forecast    = df_forecast.schema
schema_historical  = df_historical.schema
schema_target      = df_target.schema

In [6]:
pd.set_option('display.max_columns', None)
X, y = df_data.drop("target"), df_data.select("target")
X = feature_eng(X, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)

In [7]:
df_train = to_pandas(X, y)
df_train = df_train[(df_train.year>=2023) & (df_train.target.notna())].reset_index(drop=True)
df_aux = df_train.copy()

#### Nulls imputation

In [8]:
# df_train["eic_count"] = df_train["eic_count"].fillna(df_train.groupby("product_type")["eic_count"].transform("mean"))
# df_train["installed_capacity"] = df_train["installed_capacity"].fillna(df_train.groupby("product_type")["installed_capacity"].transform("mean"))

def null_imputation(df):    
    df["euros_per_mwh"] = df["euros_per_mwh"].fillna(df["euros_per_mwh"].mean())
    null_columns = ["hours_ahead_fl", "temperature_fl","dewpoint_fl","cloudcover_high_fl","cloudcover_low_fl","cloudcover_mid_fl","cloudcover_total_fl","10_metre_u_wind_component_fl","10_metre_v_wind_component_fl","direct_solar_radiation_fl","surface_solar_radiation_downwards_fl","snowfall_fl","total_precipitation_fl"]
    source_columns = ["hours_ahead", "temperature","dewpoint","cloudcover_high","cloudcover_low","cloudcover_mid","cloudcover_total","10_metre_u_wind_component","10_metre_v_wind_component","direct_solar_radiation","surface_solar_radiation_downwards","snowfall","total_precipitation"]
    for null_column, source_column in zip(null_columns, source_columns):
        df[null_column] = df[null_column].fillna(df[source_column])
    return df

df_train = null_imputation(df_train)

In [9]:
def add_dummies(df):
    dum_cols = ["county", "product_type"]
    dummies = pd.get_dummies(df[dum_cols], drop_first=True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(dum_cols, axis=1)
    return df

In [10]:
# dummies
df_train = add_dummies(df_train)

In [11]:
import holidays
est_holidays = holidays.EST()
def add_public_holidays(date):
    if date in est_holidays: return 1
    else: return 0


In [12]:
def feature_eng_cons(df):
    df['is_weekend'] = np.where(df.weekday.isin([6,7]),1,0)
    df["is_public_holiday"] = df["date"].apply(add_public_holidays)
    df = df.drop(["10_metre_u_wind_component", "10_metre_v_wind_component"], axis=1) # "hours_ahead", "hours_ahead_fl"
    return df

def feature_eng_prod(df):
    df["target_1_2_ratio"] = np.where(df['target_2'] == 0, 0, df['target_1'] / df['target_2'])
    df["capacity_per_eic"] = df["installed_capacity"] / df["eic_count"]
    df = df.drop(["highest_price_per_mwh","10_metre_u_wind_component", "10_metre_v_wind_component", "hours_ahead", "hours_ahead_fl"], axis=1)
    return df # "installed_capacity","eic_count",

In [13]:
df_cons = df_train[df_train.is_consumption==1].copy()
df_cons = feature_eng_cons(df_cons)
df_train_cons = df_cons[(df_cons.date >= pd.to_datetime("2023-05-31")- timedelta(days=50)) & (df_cons.date <= pd.to_datetime("2023-05-30"))]
df_test_cons = df_cons[df_cons.date == pd.to_datetime("2023-05-31")]

In [14]:
df_prod = df_train[df_train.is_consumption==0].copy()
df_prod = feature_eng_prod(df_prod)
df_train_prod = df_prod[(df_prod.date >= pd.to_datetime("2023-05-31") - timedelta(days=50)) & (df_prod.date <= pd.to_datetime("2023-05-30"))]
df_test_prod = df_prod[df_prod.date == pd.to_datetime("2023-05-31")]

In [15]:
from catboost import CatBoostRegressor

class Model:
    
    def __init__(self):
        self.params = {
            "n_estimators": 2000,
            "learning_rate": 0.06,
            "colsample_bytree": 0.9,
            "colsample_bynode": 0.6,
            "lambda_l1": 3.5,
            "lambda_l2": 1.5,
            "max_depth": 16,
            "num_leaves": 500,
            "min_data_in_leaf": 50,
            "objective": "regression_l1",
            # "device": "gpu",
        }
        # self.model_cons = lgb.LGBMRegressor() # **self.params
        self.model_prod = lgb.LGBMRegressor()
        self.model_cons = CatBoostRegressor(colsample_bylevel=0.878,
                    reg_lambda=3.438, learning_rate=0.1,
                   max_depth=5, min_data_in_leaf=500,
                                            verbose=False,
                   objective='MAE', random_state=1)
        
    def fit_cons(self, df_cons):
        x_train_cons = df_cons.drop(["target", "date", "is_consumption"], axis=1)
        self.cons_columns = x_train_cons.columns
        y_train_cons = df_cons["target"]
        self.model_cons.fit(x_train_cons, y_train_cons)
        return self
        
    def fit_prod(self, df_prod):
        x_train_prod = df_prod.drop(["target", "date", "is_consumption"], axis=1)
        self.prod_columns = x_train_prod.columns
        y_train_prod = df_prod["target"]
        self.model_prod.fit(x_train_prod, y_train_prod)
        return self
    
    def predict_cons(self, df_test):
        x_test = df_test.drop(["target", "date", "is_consumption"], axis=1)
        y_test = df_test["target"]
        y_pred = self.model_cons.predict(x_test)
        return mean_absolute_error(y_test, y_pred)
    
    def predict_prod(self, df_test):
        x_test = df_test.drop(["target", "date", "is_consumption"], axis=1)
        y_test = df_test["target"]
        y_pred = self.model_prod.predict(x_test)
        return mean_absolute_error(y_test, y_pred)
    
    def predict(self, df_test):
        predictions = np.zeros(len(df_test))
        mask = df_test["is_consumption"] == 1
        predictions[mask.values] = np.clip(self.model_cons.predict(df_test[mask][self.cons_columns]), 0, np.Inf)
        mask = df_test["is_consumption"] == 0
        predictions[mask.values] = np.clip(self.model_prod.predict(df_test[mask][self.prod_columns]), 0, np.Inf)
        return predictions
    

In [16]:
# 50: 40.87, 61.47

In [17]:
model = Model()

model.fit_cons(df_train_cons)
print(model.predict_cons(df_test_cons))

model.fit_prod(df_train_prod)
print(model.predict_prod(df_test_prod))

38.918486456081965
61.47812003812397


## Submission

In [18]:
import enefit
env = enefit.make_env()
iter_test = env.iter_test()

In [19]:
def is_prediciton_needed(test):
    return not all(test['currently_scored'] == False)

for (test, revealed_targets, client, historical_weather,
        forecast_weather, electricity_prices, gas_prices, sample_prediction) in iter_test:
   
    test = test.rename(columns={"prediction_datetime": "datetime"})
    
    df_test           = pl.from_pandas(test[data_cols[1:]], schema_overrides=schema_data)
    df_client         = pl.from_pandas(client[client_cols], schema_overrides=schema_client)
    df_gas            = pl.from_pandas(gas_prices[gas_cols], schema_overrides=schema_gas)
    df_electricity    = pl.from_pandas(electricity_prices[electricity_cols], schema_overrides=schema_electricity)
    df_new_forecast   = pl.from_pandas(forecast_weather[forecast_cols], schema_overrides=schema_forecast)
    df_new_historical = pl.from_pandas(historical_weather[historical_cols], schema_overrides=schema_historical)
    df_new_target     = pl.from_pandas(revealed_targets[target_cols], schema_overrides=schema_target)
    
    df_forecast       = pl.concat([df_forecast, df_new_forecast]).unique()
    df_historical     = pl.concat([df_historical, df_new_historical]).unique()
    df_target         = pl.concat([df_target, df_new_target]).unique()
    
    X_test = feature_eng(df_test, df_client, df_gas, df_electricity, df_forecast, df_historical, df_location, df_target)
    X_test = to_pandas(X_test)
    
    X_test = null_imputation(X_test)
    X_test = add_dummies(X_test)
    
    X_test_cons = X_test[X_test["is_consumption"]==1]
    X_test_prod = X_test[X_test["is_consumption"]==0]
    
    X_test_cons = feature_eng_cons(X_test_cons)
    X_test_prod = feature_eng_prod(X_test_prod)
    
    X_test_all = pd.concat([X_test_cons.drop("date", axis=1), X_test_prod.drop("date", axis=1)]).sort_index()
        
    if not is_prediciton_needed(test):
        sample_prediction['target'] = 0
        env.predict(sample_prediction)
        continue
    
    sample_prediction["target"] = model.predict(X_test_all)
    env.predict(sample_prediction)

This version of the API is not optimized and should not be used to estimate the runtime of your code on the hidden test set.


/tmp/ipykernel_20/517714714.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_weekend'] = np.where(df.weekday.isin([6,7]),1,0)
/tmp/ipykernel_20/517714714.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["is_public_holiday"] = df["date"].apply(add_public_holidays)
/tmp/ipykernel_20/517714714.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pan